<a href="https://colab.research.google.com/github/rickvelloso/Learning-IA/blob/main/MultiLabel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import hamming_loss
from sklearn.multioutput import ClassifierChain
from sklearn.preprocessing import StandardScaler

In [3]:
musica = pd.read_csv('Musica.csv')
musica.head(4)

,amazed-suprised,happy-pleased,relaxing-clam,quiet-still,sad-lonely,angry-aggresive,Mean_Acc1298_Mean_Mem40_Centroid,Mean_Acc1298_Mean_Mem40_Rolloff,Mean_Acc1298_Mean_Mem40_Flux,Mean_Acc1298_Mean_Mem40_MFCC_0,...,Std_Acc1298_Std_Mem40_MFCC_10,Std_Acc1298_Std_Mem40_MFCC_11,Std_Acc1298_Std_Mem40_MFCC_12,BH_LowPeakAmp,BH_LowPeakBPM,BH_HighPeakAmp,BH_HighPeakBPM,BHSUM1,BHSUM2,BHSUM3
0,0,1,1,0,0,0,0.132498,0.077848,0.229227,0.602629,...,0.197026,0.196244,0.164323,0.030017,0.253968,0.008473,0.240602,0.136735,0.058442,0.107594
1,1,0,0,0,0,1,0.384281,0.355249,0.167190,0.853089,...,0.093526,0.085649,0.025101,0.182955,0.285714,0.156764,0.270677,0.191377,0.153728,0.197951
2,0,1,0,0,0,1,0.541782,0.356491,0.152246,0.791142,...,0.198082,0.108067,0.140574,0.099303,0.142857,0.000000,0.593985,0.105114,0.025555,0.122965
3,0,0,1,0,0,0,0.174288,0.243935,0.254326,0.438987,...,0.235793,0.220195,0.235834,0.024988,0.222222,0.117169,0.210526,0.057288,0.134575,0.091509


In [4]:
musica.shape

(592, 77)

In [6]:
classe = musica.iloc[:,0:6].values
previsores = musica.iloc[:,7:78].values

In [7]:
scaler = StandardScaler()
previsores = scaler.fit_transform(previsores)

In [8]:
previsores

array([[-0.8981437 ,  0.59645542, -0.03960736, ..., -0.49591226,
        -0.69187266, -0.854754  ],
       [ 0.66080303,  0.00318568,  1.37433704, ..., -0.20667729,
        -0.21862043, -0.38517106],
       [ 0.66778286, -0.13972618,  1.02462206, ..., -0.66329082,
        -0.85521088, -0.77487132],
       ...,
       [-1.10473431, -1.13171275, -1.15410898, ..., -0.67620641,
        -0.29937817, -0.74471328],
       [ 0.36627331,  1.10524422,  0.26285006, ...,  0.39361683,
        -0.47519758,  0.45232124],
       [-0.46329804, -0.55724418, -0.19985176, ..., -0.96218602,
        -0.3278917 , -0.78358664]])

In [10]:
X_treinamento, X_teste, y_treinamento, y_teste = train_test_split(previsores, classe, test_size=0.3, random_state=5)

In [12]:
svm = SVC()
classifier_chain = ClassifierChain(base_estimator=svm, random_state=5)
classifier_chain.fit(X_treinamento, y_treinamento)

ClassifierChain(base_estimator=SVC(), random_state=5)

In [13]:
y_pred = classifier_chain.predict(X_teste)
loss = hamming_loss(y_teste, y_pred)
print(f"Hamming Loss: {loss}")
#0.18% das classificações ficaram erradas utilizando este modelo.

Hamming Loss: 0.18258426966292135
